<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title #####  Put a YouTube link below and hit Run a cell. Use any-language video and get any-language summary. Quick access to this colab notebook. [tinyurl.com/summarisation](https://tinyurl.com/summarisation) { display-mode: "form" }
%config InteractiveShell.ast_node_interactivity="all" #update python display config

In [2]:
# @title  Summarize { run: "auto", display-mode: "form" }
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=t6VoqbAh9HM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link
Translate_to = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3:8b-instruct-q8_0" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = True #@param {type:"boolean"}
# @markdown  * otherwise transcribe audio


#2 launch ollama server with llama3
try: # to faster next start
  import pytube
except ImportError:
  !curl -fsSL https://ollama.com/install.sh | sh
  !tmux new -d ollama serve
  !ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

def install_whisperx():
    try:
      import whisperx
    except ImportError:
      print("installing whisperx...may take a few minutes")
      !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
      clear_output()

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  install_whisperx()

try:
  import deep_translator
except ImportError:
  !pip install -U deep-translator

try:
  import openai
except ImportError:
  !pip install openai
  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)

if Type == "Youtube video or playlist":
  try:
    import pytubefix
  except ImportError:
    #!pip install git+https://github.com/pytube/pytube
    !pip install git+https://github.com/JuanBindez/pytubefix
    #clear_output()
    from pytubefix import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/gdrive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def check_if_captions_exist(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)

    try:
      transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
      return (True)
    except:
      return (False)

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id) #languages=['en']
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)

    if use_Youtube_captions and check_if_captions_exist(URL):
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      install_whisperx()
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', '/gdrive/My Drive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt including a concise title that reflects the content. Wrap the title with **markdown bold notation**. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]



def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time
from deep_translator import GoogleTranslator

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": summary_prompt},
            {"role": "user", "content": prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text,chunk_size):
    overlap_size = 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    final_summary =""
    if Translate_to != "English":
      translated_summaries = []
      for _,summary in summaries:
        translated_summary = GoogleTranslator(source='auto', target=language_codes[Translate_to]).translate(summary)
        translated_summaries.append(translated_summary)
        final_summary += translated_summary + "\n\n"
      final_summary = final_summary.strip()
    else :
      final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL_"+str(chunk_size)+".md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)
    return (final_summary)

from IPython.core.interactiveshell import InteractiveShell # Unblock output
InteractiveShell.ast_node_interactivity = "all"            # It starts to work after restart cell
#%config InteractiveShell.ast_node_interactivity="all"
clear_output()
print ("Generating summary...")
general_summary = process_and_summarize(text,15000) # max string size = max_tokens x 4
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary)
final_summary = process_and_summarize(text,3072)
clear_output()
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary + "\n\n" + "\n\n **Long Summary:** \n\n" + final_summary )





 **Short Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 It seems like you're creating a website about drone reviews and comparisons, and you're using AI tools like ChatGPT and Claude to generate content. You started by writing a prompt for Claude to create a heading and subheading in the style of Alex Horoi and David Ogoli, and it generated a heading and subheading that you can use on your website.

You then talked about generating images using Midjourney and Dolly 3, and how you can tweak settings to get the desired output. You created an image of a drone flying above a city skyline and replaced an existing image on your website with the new one.

Finally, you mentioned that you'll be using Canva to create featured images for blog posts, as it's easy to use and allows you to add text over images and remove backgrounds.

It sounds like you're using AI tools to streamline the content creation process for your website. By providing clear prompts and guidance, you can get high-quality output from these tools that helps you build a professional-looking website.


00:11:15 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=675 **AI-Powered Website Creation: A Step-by-Step Guide**

The speaker continues to demonstrate how they used various AI tools to create a website, specifically focusing on featured images. They use Canva's Magic Media feature to generate images, edit them using AI-powered background remover, and add text and effects to make it look professional. The speaker also shares their experience with other AI tools such as Reloom for site mapping, ColorMind for color schemes, FontJoy for font pairings, and chatGPT or Claude for writing copy.




 **Long Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **AI-Powered Website Creation Tools**

You mentioned needing a website, whether it's for sharing thoughts or selling products. There are AI-enabled tools that can help you get online quickly and easily without any technical knowledge required. One such tool is Reloom, which uses AI to map out your entire website. It has a free plan forever for simple websites and more premium plans for those using design tools like Web Flow and Figma. We're going to start with the free plan and explore what Reloom can do.


00:02:18 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=138 **Building a Website with AI**

So, we're already halfway through setting up our website. We've got hosting taken care of with Hostinger, and they're offering a sweet deal for our viewers. With their Business plan, we can get our website built by AI on WordPress in just minutes. We selected the Business website builder since most of the AI tools are included under this plan. Now, let's create our website and start editing it.


00:04:34 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=274 **Using AI-powered Tools to Create a Website Color Scheme**

So, we've got a color palette generated from Colormind, an AI-powered tool that uses machine learning to find colors that work together. We liked the blue and locked it in, then generated more variations until we found one that looks good. Now we can copy the codes and use them on our website.

Next, we're going to explore fonts using Font Joy, another AI site that recommends font pairings. We sat here and generated a few different font styles until we found a font we really like - Robotto Condensed. We locked it in and generated again until it suggested a couple of fonts that look good with our chosen font. Now we can go back to our website and select the fonts we want to use, using Roboto Condensed for headers and Work Sans for subheadings.


00:06:48 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=408 **AI-Powered Content Creation**

So, as I was saying, now that we've got our site map, color scheme, and font pairings set up, it's time to start filling in some content. For this, I'm going to use AI tools like ChatGPT or Claude to generate the content. Since both models are using similar technology, the output will likely be similar for copywriting tasks. The key is crafting a prompt that gets the desired style of writing. In my case, I'm creating a website that reviews drones and wants to write engaging copy to encourage visitors to use the site.


00:09:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=541 **Creating Visual Content for Your Website**

We're copying our heading and subheading from another source, replacing the current text with our own. We'll need to repeat this step for each section of our website. The next step is selecting images that are relevant to our content. I like how they used a DJI Mavic 2 in one of their images, but we'll want more variety throughout the site. There are several tools available to generate images, including Mid Journey and Dolly 3. Let's try creating an image of a camera drone flying above a city skyline using Mid Journey. We can tweak some settings to get a more realistic image, and then download it to use on our website. I'll replace the current image with one we just created, making sure it's centered and relevant to our content. Finally, let's discuss creating featured images for blog posts using Canva, which is easy to use and allows us to add text, remove backgrounds, and more.


00:11:27 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=687 **Using Canva's Image Generator and Editing Tools**

As we were saying, we're looking to create a consistent style for our blog featured images. We can use Canva's Magic Media feature, which has an image generator that allows us to create images or graphics with a specific style. We experimented with different styles, including filmic and camera drone flying above a city skyline. The generated images were decent, so we selected a few and started editing them.


00:13:31 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=811 **Building a Website with AI in Minutes**

So, we've got our featured image all set up using Canva and various AI features. We added some effects to make it pop off the screen, and now it's looking pretty good. I'll go ahead and save it as a JPEG and download it to use on our website.


00:15:55 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=955 **Curating Cool AI Tools and Giving Away Free Gadgets**

The speaker wraps up by promoting his channel's newsletter, Future Tools, where he curates a daily list of cool AI tools and news. He also announces that every month, he'll be giving away free gadgets to subscribers who sign up for the newsletter and are also subscribed to his YouTube channel.
